<a href="https://colab.research.google.com/github/mccetrangolo/trabalho_final_ia/blob/main/relatorio_trabalho_IA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#trabalho realizado pelos aluno Gustavo Francisco S. S. França,Mauricio Cesar Cetrangolo e Everton Ferreira de Lima
!pip -q install -U pandas scikit-learn fairlearn shap tensorflow


In [17]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

from fairlearn.metrics import demographic_parity_difference, demographic_parity_ratio, equal_opportunity_difference
from fairlearn.adversarial import AdversarialFairnessClassifier

import tensorflow as tf
import shap


# -----------------------------
# Helpers
# -----------------------------
def to_dense(X):
    return X.toarray() if hasattr(X, "toarray") else np.asarray(X)


def build_preprocess(X: pd.DataFrame) -> ColumnTransformer:
    cat_cols = [c for c in X.columns if X[c].dtype == "object" or str(X[c].dtype).startswith("category")]
    num_cols = [c for c in X.columns if c not in cat_cols]

    numeric = Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler()),
    ])
    categorical = Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("onehot", OneHotEncoder(handle_unknown="ignore")),
    ])

    return ColumnTransformer(
        transformers=[("num", numeric, num_cols), ("cat", categorical, cat_cols)],
        remainder="drop",
        verbose_feature_names_out=False,
    )


def perf_metrics(y_true, y_prob, threshold=0.5) -> dict:
    y_hat = (y_prob >= threshold).astype(int)
    return {
        "accuracy": float(accuracy_score(y_true, y_hat)),
        "f1": float(f1_score(y_true, y_hat, zero_division=0)),
        "auc": float(roc_auc_score(y_true, y_prob)) if len(np.unique(y_true)) > 1 else float("nan"),
    }


def make_tf_models(input_dim: int):
    predictor = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(input_dim,)),
        tf.keras.layers.Dense(32, activation="relu"),
        tf.keras.layers.Dense(1)
    ])
    adversary = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(1,)),
        tf.keras.layers.Dense(16, activation="relu"),
        tf.keras.layers.Dense(1)
    ])
    return predictor, adversary


def make_binary_target(y_raw: pd.Series, dataset_name: str) -> np.ndarray:
    y_raw = pd.Series(y_raw)

    if pd.api.types.is_numeric_dtype(y_raw):
        y_num = pd.to_numeric(y_raw, errors="coerce")
        if y_num.nunique(dropna=True) < 2:
            raise ValueError(f"[{dataset_name}] Target numérico tem só 1 classe: {y_num.unique()}")
        return (y_num > np.nanmin(y_num)).astype(int).to_numpy()

    y_str = y_raw.astype(str).str.strip().str.lower()
    positives = {"yes", "y", "true", "1", "pos", "positive", "success", "default", ">50k", ">50k."}
    y_bin = y_str.isin(positives).astype(int).to_numpy()

    if len(np.unique(y_bin)) < 2:
        counts = y_str.value_counts(dropna=False)
        if len(counts) < 2:
            raise ValueError(f"[{dataset_name}] Target tem só 1 valor: {counts.index.tolist()}")
        positive_label = counts.index[-1]  # least frequent
        print(f"[{dataset_name}] Fallback: usando como classe positiva: {positive_label}")
        y_bin = (y_str == positive_label).astype(int).to_numpy()

    return y_bin


def binarize_by_median(series: pd.Series, name: str):
    v = pd.to_numeric(series, errors="coerce")
    med = np.nanmedian(v)
    sens = np.where(v >= med, f"{name}>=mediana", f"{name}<mediana")
    return sens


def run_dataset(name: str, X: pd.DataFrame, y_bin: np.ndarray, sensitive: np.ndarray,
                sensitive_name: str, out_csv: str):
    print("\n==============================")
    print("DATASET:", name)
    print("==============================\n")
    print("Atributo sensível:", sensitive_name)
    print("Distribuição do atributo sensível:", pd.Series(sensitive).value_counts(dropna=False).to_dict())
    print("Distribuição do target:", pd.Series(y_bin).value_counts(dropna=False).to_dict())

    if pd.Series(y_bin).nunique() < 2:
        raise ValueError(f"[{name}] Target binário ficou com 1 classe só. Verifique o mapeamento.")

    X_train, X_test, y_train, y_test, s_train, s_test = train_test_split(
        X, y_bin, sensitive, test_size=0.2, random_state=42, stratify=y_bin
    )

    pre = build_preprocess(X_train)
    pre.fit(X_train)
    Xtr = pre.transform(X_train)
    Xte = pre.transform(X_test)

    base = LogisticRegression(max_iter=2000)
    base.fit(Xtr, y_train)
    y_prob_base = base.predict_proba(Xte)[:, 1]

    Xtr_d = to_dense(Xtr)
    Xte_d = to_dense(Xte)

    predictor, adversary = make_tf_models(input_dim=Xtr_d.shape[1])
    adv = AdversarialFairnessClassifier(
        predictor_model=predictor,
        adversary_model=adversary,
        constraints="demographic_parity",
        alpha=0.01,
        learning_rate=0.001,
        epochs=20,
        batch_size=256,
        random_state=42,
    )
    adv.fit(Xtr_d, y_train, sensitive_features=s_train)

    if hasattr(adv, "predict_proba"):
        y_prob_adv = adv.predict_proba(Xte_d)[:, 1]
    elif hasattr(adv, "decision_function"):
        scores = adv.decision_function(Xte_d)
        y_prob_adv = 1.0 / (1.0 + np.exp(-scores))
    else:
        y_prob_adv = adv.predict(Xte_d).astype(float)

    thr = 0.5
    yhat_base = (y_prob_base >= thr).astype(int)
    yhat_adv  = (y_prob_adv  >= thr).astype(int)

    perf_base = perf_metrics(y_test, y_prob_base, threshold=thr)
    perf_adv  = perf_metrics(y_test, y_prob_adv, threshold=thr)

    fair_base = {
        "DPD": float(demographic_parity_difference(y_test, yhat_base, sensitive_features=s_test)),
        "DI":  float(demographic_parity_ratio(y_test, yhat_base, sensitive_features=s_test)),
        "EOD": float(equal_opportunity_difference(y_test, yhat_base, sensitive_features=s_test)),
    }
    fair_adv = {
        "DPD": float(demographic_parity_difference(y_test, yhat_adv, sensitive_features=s_test)),
        "DI":  float(demographic_parity_ratio(y_test, yhat_adv, sensitive_features=s_test)),
        "EOD": float(equal_opportunity_difference(y_test, yhat_adv, sensitive_features=s_test)),
    }

    print("\n=== BASELINE (LogReg) ===")
    print(perf_base)
    print(fair_base)

    print("\n=== ADVERSARIAL (Fairlearn) ===")
    print(perf_adv)
    print(fair_adv)

    feature_names = list(pre.get_feature_names_out())

    expl_base = shap.LinearExplainer(base, Xtr, feature_perturbation="interventional")
    shap_base = expl_base.shap_values(Xte[:200])
    imp_base = np.mean(np.abs(shap_base), axis=0)

    X_bg = Xtr_d[:100]
    X_ex = Xte_d[:100]

    def adv_proba(Xarr):
        if hasattr(adv, "predict_proba"):
            return adv.predict_proba(Xarr)[:, 1]
        if hasattr(adv, "decision_function"):
            sc = adv.decision_function(Xarr)
            return 1.0 / (1.0 + np.exp(-sc))
        return adv.predict(Xarr).astype(float)

    expl_adv = shap.KernelExplainer(adv_proba, X_bg)
    shap_adv = expl_adv.shap_values(X_ex, nsamples=100)
    if isinstance(shap_adv, list):
        shap_adv = shap_adv[0]
    imp_adv = np.mean(np.abs(shap_adv), axis=0)

    df_imp = pd.DataFrame({
        "feature": feature_names,
        "mean_abs_shap_baseline": imp_base,
        "mean_abs_shap_adversarial": imp_adv,
        "delta": imp_adv - imp_base
    }).sort_values("mean_abs_shap_baseline", ascending=False)

    df_imp.to_csv(out_csv, index=False)
    print(f"\n[OK] SHAP salvo em: {out_csv}")
    print("\nTop-10 features (baseline):")
    print(df_imp.head(10).to_string(index=False))

    return perf_base, fair_base, perf_adv, fair_adv


# -----------------------------
# Load datasets and run all
# -----------------------------
summary = []

# 1) ADULT
from fairlearn.datasets import fetch_adult
X_adult, y_adult = fetch_adult(return_X_y=True, as_frame=True)
y_adult_bin = make_binary_target(pd.Series(y_adult), "Adult")
s_adult = X_adult["sex"].to_numpy()

pb, fb, pa, fa = run_dataset(
    "Adult (Census Income)", X_adult, y_adult_bin, s_adult, "sex", "adult_shap_importance.csv"
)
summary.append(("Adult", pb, fb, pa, fa))

# 2) BANK (V1..V16)
from fairlearn.datasets import fetch_bank_marketing
X_bank, y_bank = fetch_bank_marketing(return_X_y=True, as_frame=True)
y_bank_bin = make_binary_target(pd.Series(y_bank), "Bank Marketing")

# sensível = V1 binarizado
if "V1" not in X_bank.columns:
    raise ValueError(f"Bank: esperado V1..Vn. Colunas: {list(X_bank.columns)}")
s_bank = binarize_by_median(X_bank["V1"], "V1")

pb, fb, pa, fa = run_dataset(
    "Bank Marketing (Term Deposit)", X_bank, y_bank_bin, s_bank, "V1_bin(mediana)", "bank_shap_importance.csv"
)
summary.append(("Bank", pb, fb, pa, fa))

# 3) CREDIT (x1..x23)
from fairlearn.datasets import fetch_credit_card
X_credit, y_credit = fetch_credit_card(return_X_y=True, as_frame=True)
y_credit_bin = make_binary_target(pd.Series(y_credit), "Credit Card Default")

# escolher coluna sensível (x2 preferencial, fallback x1)
cols = [str(c).strip() for c in X_credit.columns]
X_credit.columns = cols

sens_col = "x2" if "x2" in X_credit.columns else ("x1" if "x1" in X_credit.columns else None)
if sens_col is None:
    raise ValueError(f"Credit: esperado x1..xN. Colunas: {list(X_credit.columns)}")

s_credit = binarize_by_median(X_credit[sens_col], sens_col)

pb, fb, pa, fa = run_dataset(
    "Default of Credit Card Clients", X_credit, y_credit_bin, s_credit, f"{sens_col}_bin(mediana)", "credit_shap_importance.csv"
)
summary.append(("Credit", pb, fb, pa, fa))

print("\n\n=== RESUMO (para preencher tabelas) ===")
for name, perf_b, fair_b, perf_a, fair_a in summary:
    print("\n---", name, "---")
    print("BASELINE perf:", perf_b)
    print("BASELINE fair:", fair_b)
    print("ADV perf:", perf_a)
    print("ADV fair:", fair_a)



DATASET: Adult (Census Income)

Atributo sensível: sex
Distribuição do atributo sensível: {'Male': 32650, 'Female': 16192}
Distribuição do target: {0: 37155, 1: 11687}

=== BASELINE (LogReg) ===
{'accuracy': 0.8523902139420616, 'f1': 0.6561754887935145, 'auc': 0.9042148127759706}
{'DPD': 0.17136809845735007, 'DI': 0.3066461647250784, 'EOD': 0.09651434712490026}

=== ADVERSARIAL (Fairlearn) ===
{'accuracy': 0.8456341488381616, 'f1': 0.6087182148417228, 'auc': 0.7277764385871546}
{'DPD': 0.05697967910204001, 'DI': 0.6728944347845851, 'EOD': 0.17978918224400547}


  0%|          | 0/100 [00:00<?, ?it/s]


[OK] SHAP salvo em: adult_shap_importance.csv

Top-10 features (baseline):
                          feature  mean_abs_shap_baseline  mean_abs_shap_adversarial     delta
marital-status_Married-civ-spouse                0.831662                   0.003019 -0.828643
                    education-num                0.594410                   0.025652 -0.568759
                     capital-gain                0.588834                   0.038368 -0.550466
     marital-status_Never-married                0.554016                   0.010381 -0.543635
                       sex_Female                0.397437                   0.013229 -0.384207
                              age                0.259329                   0.012052 -0.247277
                   hours-per-week                0.226779                   0.011491 -0.215288
           relationship_Own-child                0.206963                   0.003086 -0.203878
             relationship_Husband                0.206613            

  0%|          | 0/100 [00:00<?, ?it/s]


[OK] SHAP salvo em: bank_shap_importance.csv

Top-10 features (baseline):
    feature  mean_abs_shap_baseline  mean_abs_shap_adversarial     delta
        V12                0.750248                   0.074905 -0.675343
 V9_unknown                0.464315                   0.026596 -0.437720
     V7_yes                0.290611                   0.012418 -0.278193
    V11_jul                0.287215                   0.010824 -0.276391
V16_unknown                0.247470                   0.019627 -0.227843
    V11_may                0.197997                   0.016213 -0.181784
V9_cellular                0.186395                   0.007591 -0.178803
V16_success                0.170747                   0.002500 -0.168248
    V11_aug                0.169461                   0.008580 -0.160881
 V3_married                0.169200                   0.019826 -0.149374

DATASET: Default of Credit Card Clients

Atributo sensível: x2_bin(mediana)
Distribuição do atributo sensível: {'x2>=medi

  0%|          | 0/100 [00:00<?, ?it/s]


[OK] SHAP salvo em: credit_shap_importance.csv

Top-10 features (baseline):
feature  mean_abs_shap_baseline  mean_abs_shap_adversarial     delta
     x6                0.481530                   0.082533 -0.398997
    x12                0.232072                   0.015244 -0.216828
     x1                0.110672                   0.019124 -0.091548
    x18                0.090140                   0.010216 -0.079925
     x8                0.080132                   0.026446 -0.053686
     x3                0.075738                   0.012321 -0.063416
     x7                0.074174                   0.020709 -0.053466
     x4                0.070444                   0.010825 -0.059620
    x13                0.067444                   0.039419 -0.028025
     x5                0.063700                   0.018662 -0.045038


=== RESUMO (para preencher tabelas) ===

--- Adult ---
BASELINE perf: {'accuracy': 0.8523902139420616, 'f1': 0.6561754887935145, 'auc': 0.9042148127759706}
BASELI

In [18]:
from google.colab import files
files.download("adult_shap_importance.csv")
files.download("bank_shap_importance.csv")
files.download("credit_shap_importance.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>